In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from numpy import random
import random
from pprint import pprint

In [2]:
data=pd.read_csv('D:\\Data_sets\\titanic.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df=data[['Pclass','Sex','Survived']]

In [4]:
df

,Pclass,Sex,Survived
0,3,male,0
1,1,female,1
2,3,female,1
3,1,female,1
4,3,male,0
...,...,...,...
886,2,male,0
887,1,female,1
888,3,female,0
889,1,male,1


In [5]:
df.dtypes

Pclass       int64
Sex         object
Survived     int64
dtype: object

In [6]:
df.isnull().sum()

Pclass      0
Sex         0
Survived    0
dtype: int64

In [26]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
#     print(indices)
    
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [27]:
random.seed(0)
train_df, test_df = train_test_split(df, test_size=0.2)

In [28]:
train_df.shape

(713, 3)

In [29]:
test_df.shape

(178, 3)

In [30]:
train_df.columns

Index(['Pclass', 'Sex', 'Survived'], dtype='object')

In [31]:
for i in train_df.columns:
    print(i,len(df[i].unique()))

Pclass 3
Sex 2
Survived 2


In [32]:
## Determine the type of variable

In [33]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            
#             print(isinstance(unique_values[0],str))
            
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [34]:
determine_type_of_feature(train_df)

['categorical', 'categorical', 'categorical']

In [35]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [36]:
check_purity(train_df.values)

False

In [37]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [38]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    
  
    data_below = data[split_column_values == split_value]
    data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [39]:
# df.values[:, 0]

In [40]:
b,a=split_data(train_df.values,split_column=0, split_value=2)

In [41]:
np.unique(b[:,0])

array([2], dtype=object)

In [42]:
np.unique(a[:,0])

array([1, 3], dtype=object)

In [43]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        print('length',len(unique_values))
        
        if len(unique_values)>1:
            potential_splits[column_index] = unique_values
    
        potential_splits[column_index] = unique_values
        
    return potential_splits

In [44]:
train_df.values

array([[3, 'male', 0],
       [3, 'female', 1],
       [1, 'female', 1],
       ...,
       [3, 'female', 0],
       [2, 'male', 0],
       [1, 'female', 1]], dtype=object)

In [45]:
get_potential_splits(train_df.values)

length 3
length 2


{0: array([1, 2, 3], dtype=object), 1: array(['female', 'male'], dtype=object)}

In [46]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [47]:
calculate_entropy(train_df.values)

0.9590852514215054

In [48]:
def entropy(sample):
    
    probabilities=[]
    for i in sample:
        prob=i/sum(sample)
        probabilities.append(prob)
        
    
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [49]:
train_df.Survived.value_counts()

0    441
1    272
Name: Survived, dtype: int64

In [50]:
entropy([39,37,44])

1.5810951599090195

In [51]:
def calculate_overall_entropy(data_below, data_above):
    
#     print(data_below)
#     print('**********************************************')
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n
    
    
    print(calculate_entropy(data_below))
    print('------------------------')
    print(calculate_entropy(data_above))

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [52]:
def determine_best_split(data, potential_splits):
    
#     print(data)
    
#     print(potential_splits)
    
#     small=[]
    
    overall_entropy = 9999
    for column_index in potential_splits:
        
        
        
        for value in potential_splits[column_index]:
            
            
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
#             print('split value=',np.unique(data_below[:,column_index]), np.unique(data_above[:,column_index]))
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
            
            
            
            unique, counts1 = np.unique(data_below[:,2], return_counts=True)
            unique, counts2 = np.unique(data_above[:,2], return_counts=True)
    
            print(counts1, counts2)
            
            
#             small.append([column_index,value,current_overall_entropy])
            
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
            print('overall entropy=',overall_entropy)
            print('*************************************************************')
                
                
    print('column split=',best_split_column, best_split_value)
    print('selected #######################################################',current_overall_entropy)
    
#     print(small)
    
    
    
    return best_split_column, best_split_value

In [53]:
determine_best_split(train_df.values, get_potential_splits(train_df.values))

length 3
length 2
0.9627730052387542
------------------------
0.8932793316597367
[ 65 103] [376 169]
overall entropy= 0.9096537175801784
*************************************************************
1.0
------------------------
0.9353951178931705
[72 72] [369 200]
overall entropy= 0.9096537175801784
*************************************************************
0.7981778534399544
------------------------
0.9892729241947813
[304  97] [137 175]
overall entropy= 0.881798697865629
*************************************************************
0.8112781244591328
------------------------
0.6801719444263874
[ 63 189] [378  83]
overall entropy= 0.7265096125445527
*************************************************************
0.6801719444263874
------------------------
0.8112781244591328
[378  83] [ 63 189]
overall entropy= 0.7265096125445527
*************************************************************
column split= 1 male
selected ####################################################### 0.7265096

(1, 'male')

In [54]:
FEATURE_TYPES = determine_type_of_feature(train_df)
FEATURE_TYPES

['categorical', 'categorical', 'categorical']

In [55]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
#         print(potential_splits)
        
        if len(potential_splits)==0:
            classification=classify_data(data)
            return classification
        
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
#         print(data_below)
        
        # check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [56]:
tree=decision_tree_algorithm(train_df,max_depth=5)
pprint(tree, width=10)

length 3
length 2
0.9627730052387542
------------------------
0.8932793316597367
[ 65 103] [376 169]
overall entropy= 0.9096537175801784
*************************************************************
1.0
------------------------
0.9353951178931705
[72 72] [369 200]
overall entropy= 0.9096537175801784
*************************************************************
0.7981778534399544
------------------------
0.9892729241947813
[304  97] [137 175]
overall entropy= 0.881798697865629
*************************************************************
0.8112781244591328
------------------------
0.6801719444263874
[ 63 189] [378  83]
overall entropy= 0.7265096125445527
*************************************************************
0.6801719444263874
------------------------
0.8112781244591328
[378  83] [ 63 189]
overall entropy= 0.7265096125445527
*************************************************************
column split= 1 male
selected ####################################################### 0.7265096

In [57]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":  # feature is continuous
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)